<a href="https://colab.research.google.com/github/SeohyeonSunny/Crawling/blob/main/Naver_%EB%84%A4%EC%9D%B4%EB%B2%84_API_%EB%89%B4%EC%8A%A4_%ED%81%AC%EB%A1%A4%EB%9F%AC_%EC%A0%9C%EB%AA%A9%2C_%EB%B3%B8%EB%AC%B8_%EB%93%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=c6ea4e392022a08a2ea73992831ef5e1b4054326b50878795e21b6daf79ff630
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=3735bb8b21198746e674a473d11db4e34afff18717a2f37aa60d83d6db3b3f98
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc

In [ ]:
import requests
import pandas as pd
import datetime
from newspaper import Article
from google.colab import files

# 네이버 API 인증 정보
client_id = 'XXMEwj792bdBt4McfteH'
client_secret = 'aQz0j2Ro4F'

# 검색어와 기간 설정
query = '난민'
start_date = '2010-01-01'
end_date = '2023-12-31'

# 날짜 변환 함수
def format_date(date):
    return date.strftime('%Y-%m-%d')

# 결과를 저장할 리스트 초기화
results = []

# 날짜 범위 설정
start_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')
delta = datetime.timedelta(days=1)

# 날짜별로 뉴스 데이터 수집
current_dt = start_dt
while current_dt <= end_dt:
    current_date_str = format_date(current_dt)
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display=100&sort=date&start=1&start_date={current_date_str}&end_date={current_date_str}"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        for item in data['items']:
            title = item['title']
            description = item['description']
            pub_date = item['pubDate']
            originallink = item['originallink']
            link = item['link']

            # 신문사 정보를 추출하기 위한 초기값
            newspaper = ''

            try:
                # newspaper3k를 이용하여 원본 링크에서 신문사 정보 추출
                article = Article(originallink)
                article.download()
                article.parse()
                newspaper = article.source_url
            except:
                newspaper = 'Unknown'

            # 결과를 리스트에 추가
            results.append([title, description, pub_date, originallink, link, newspaper])
    current_dt += delta

# DataFrame으로 변환
df = pd.DataFrame(results, columns=['Title', 'Description', 'PubDate', 'OriginalLink', 'Link', 'Newspaper'])

# CSV 파일로 저장
csv_filename = 'naver_news_refugees.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

# CSV 파일 다운로드
files.download(csv_filename)